In [34]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score

In [35]:
def calculate_ece(confidences, accuracies, n_bins=10) -> float:
    """
    Calculate the expected calibration error (ECE) given a list of confidence scores (0-1) and accuracy scores (0 or 1).
    """
    df = pd.DataFrame({"conf": confidences, "acc": accuracies}).dropna()

    confidences = torch.tensor(df["conf"].tolist())
    accuracies = torch.tensor(df["acc"].tolist())
    bin_boundaries = torch.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    ece = torch.zeros(1)
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # Calculated |confidence - accuracy| in each bin
        in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
        prop_in_bin = in_bin.float().mean()
        if prop_in_bin.item() > 0:
            accuracy_in_bin = accuracies[in_bin].float().mean()
            avg_confidence_in_bin = confidences[in_bin].mean()
            ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
    
    return ece.item()

In [36]:
nan_indices = set()

for file in os.listdir("../results"):
    if file.endswith("csv"):
        df = pd.read_csv("../results/" + file)

        # Find indices where either column has NaN
        indices_with_nan = df[
            df["extracted_answer"].isna() | df["verbal_numerical_confidence"].isna()
        ].index

        # Combine indices across all files
        nan_indices.update(indices_with_nan)

# Convert to sorted list if needed
nan_indices = sorted(nan_indices)
len(nan_indices)

12032

# Drop NA on an Individual Basis

In [ ]:
stats_df = pd.DataFrame()
for file in os.listdir("../results"):
    if file.endswith("csv"):
        df = pd.read_csv("../results/" + file)
        acc_no_na = df[["extracted_answer", "correct_answer"]].dropna()
        new_row = pd.DataFrame({
            "Model": [file.replace("mmlu_pro_", "").replace("_eval_all_None.csv", "").strip().capitalize()],
            "Missing answer": [df["extracted_answer"].isna().sum()],
            "Missing verbal numerical confidence": [df["verbal_numerical_confidence"].isna().sum()],
            "Missing logit perplexity confidence": [df["logit_perplexity_confidence"].isna().sum()],
            "Missing verbal linguistic confidence": [df["verbal_linguistic_confidence"].isna().sum()],
            "Accuracy": [(df["extracted_answer"] == df["correct_answer"]).mean()],
            "Accuracy without na": [(acc_no_na["extracted_answer"] == acc_no_na["correct_answer"]).mean()],
            "ECE verbal numerical confidence": [calculate_ece(df["verbal_numerical_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE logit perplexity confidence": [calculate_ece(df["logit_perplexity_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE verbal linguistic confidence": [calculate_ece(df["verbal_linguistic_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "AUROC verbal numerical confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]).values, df["verbal_numerical_confidence"].fillna(0).values),
            "AUROC logit perplexity confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["logit_perplexity_confidence"].fillna(0).values),
            "AUROC verbal linguistic confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["verbal_linguistic_confidence"].fillna(0).values),
            "Mean verbal numerical confidence": [df["verbal_numerical_confidence"].mean()],
            "Mean logit perplexity confidence": [df["logit_perplexity_confidence"].mean()],
            "Mean verbal linguistic confidence": [df["verbal_linguistic_confidence"].mean()],
            "Std verbal numerical confidence": [df["verbal_numerical_confidence"].std()],
            "Std logit perplexity confidence": [df["logit_perplexity_confidence"].std()],
            "Std verbal linguistic confidence": [df["verbal_linguistic_confidence"].std()],
        })
        stats_df = pd.concat([stats_df, new_row], ignore_index=True)

    
display(stats_df.sort_values(by="Model", ignore_index=True))

/mnt/afs/intern/fangwenhan/miniconda3/envs/llm-uncertainty/lib/python3.13/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/mnt/afs/intern/fangwenhan/miniconda3/envs/llm-uncertainty/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/mnt/afs/intern/fangwenhan/miniconda3/envs/llm-uncertainty/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/mnt/afs/intern/fangwenhan/miniconda3/envs/llm-uncertainty/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/mnt/afs/intern/fangwenhan/miniconda3/envs/llm-uncertainty/

,Model,Missing answer,Missing verbal numerical confidence,Missing logit perplexity confidence,Missing verbal linguistic confidence,Accuracy,Accuracy without na,ECE verbal numerical confidence,ECE logit perplexity confidence,ECE verbal linguistic confidence,AUROC verbal numerical confidence,AUROC logit perplexity confidence,Mean verbal numerical confidence,Mean logit perplexity confidence,Mean verbal linguistic confidence,Std verbal numerical confidence,Std logit perplexity confidence,Std verbal linguistic confidence
0,Gemma-2-27b-it,30,0,0,0,0.531666,0.532995,0.379636,0.352674,0.000000,0.612014,0.523690,0.909806,0.884340,0.0,0.141204,0.046461,0.0
1,Gemma-2b-it,1270,668,0,0,0.133062,0.148764,0.777028,0.761161,0.000000,0.511432,0.528202,35.249272,0.894223,0.0,3406.079319,0.051987,0.0
2,Gemma-3-12b-it,12032,12032,12032,0,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0
3,Gemma-3-1b-it,2473,1214,0,0,0.138049,0.173763,0.793448,0.743742,0.000000,0.587881,0.445034,0.939211,0.881791,0.0,0.180571,0.056061,0.0
4,Gemma-3-27b-it,12032,12032,12032,0,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0
5,Gemma-3-4b-it,12032,12032,12032,0,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0
6,Llama-3.2-3b-instruct,214,196,0,0,0.311420,0.317059,0.535822,0.483946,0.000000,0.597997,0.509987,0.848269,0.795365,0.0,0.173768,0.066231,0.0
7,Meta-llama-3.1-405b-instruct-turbo_batch,114,111,0,466,0.715010,0.721849,0.164731,0.160528,0.218436,0.721472,0.608945,0.885023,0.875538,inf,0.119135,0.044896,NaN
8,Mistral-7b-instruct-v0.1,2151,883,0,0,0.214927,0.261714,0.700416,0.661195,0.000000,0.544028,0.438903,0.926236,0.876122,0.0,0.201095,0.048418,0.0
9,Mistral-nemo-instruct-2407,80,11,0,0,0.402094,0.404786,0.580863,0.449232,0.000000,0.523301,0.509669,0.983325,0.851326,0.0,0.060344,0.049183,0.0
